In [18]:
import numpy as np
from sklearn.cross_validation import train_test_split
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
import pandas as pd
from keras.utils import to_categorical
%matplotlib inline

In [2]:
data=pd.read_csv('fer2013.csv', sep=',',header=0)

In [8]:
X=np.array(data['pixels'])
Y=np.array(data['emotion'])

In [9]:
n=X.shape[0]
Xi=[]
for i in range(0,n):
    temp=[]
    for j in X[i].split():
        temp.append(int(i))
    Xi.append(temp)

In [10]:
Xnp=np.array(Xi).reshape(-1,48,48,1)

In [19]:
Y=to_categorical(Y)

In [20]:
X_train,X_test,y_train,y_test=train_test_split(Xnp,Y)

In [12]:
X_train.shape

(26915, 48, 48, 1)

In [14]:
''' Building model ''' 

input_shape=(48,48,1)
X_input = Input(input_shape)

# Zero-Padding: adding zero padding
X = ZeroPadding2D((3, 3))(X_input)

# CONV -> BN -> RELU Block applied to X
X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
X = BatchNormalization(axis = 3, name = 'bn0')(X)
X = Activation('relu')(X)

# MAXPOOL
X = MaxPooling2D((2, 2), name='max_pool')(X)

# FLATTEN X and give it to dense layer
X = Flatten()(X)
X = Dense(7, activation='sigmoid', name='fc')(X)

# Create model
model = Model(inputs = X_input, outputs = X, name='HappyModel')

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 48, 1)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 54, 54, 1)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 48, 48, 32)        1600      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 48, 48, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 24, 24, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
__________

In [16]:
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=["accuracy"])

In [21]:
model.fit(X_train,y_train,epochs=1,validation_data=(X_test,y_test))

Train on 26915 samples, validate on 8972 samples
Epoch 1/1
26915/26915 [==============================] - 239s 9ms/step - loss: 0.5022 - acc: 0.8396 - val_loss: 0.5023 - val_acc: 0.8070


In [22]:
model.save('my_model.h5')

In [23]:
np.save('pixedarray',Xi)